In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_A3_018, input_features_A3_018 = pre_models.Input_A3_018('weights/')
    
    outputs = ['Input_A3_018']

    preprocessings = ['min_max_normalization']

    models = [model_A3_018]

    input_features = [input_features_A3_018]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=4
data=pd.read_csv('./BayesianRidge_Pre_0/Input_A3_018.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_A3_019', 'Input_A1_018', 'Input_A2_019', 'Input_A5_019'],
      dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.009073277470677316
[0.009317862103355057, 0.008138360016520824, 0.009145002760415136, 0.009642481335466217, 0.009143559740722501, 0.009858064735333355, 0.00873608184880011, 0.009595900760155195, 0.007817862753020322, 0.00933759865298445]


test_r2_ave: 0.5143196935561831
[0.5087209645384301, 0.6400287045727837, 0.5132426034249755, 0.4592406280757133, 0.5242147931988762, 0.37067350735555515, 0.529088440624419, 0.48192020377046263, 0.6231708697522372, 0.49289622024837876]


## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,4)

train_rmse: 0.00872477114064293
test_rmse: 0.008856737530801326
test_r2: 0.5399154298217665


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,5e-06)

test_rmse_ave: 0.008961732550173195
[0.009007021122923667, 0.00847827463379755, 0.008882503469321276, 0.009459867700218932, 0.00959738986945189, 0.009370382468031763, 0.008393444503329482, 0.009263346279254675, 0.007919152454530228, 0.009245943000872505]


test_r2_ave: 0.5276465264309236
[0.5409520669225529, 0.6093309233020796, 0.5407854401117311, 0.4795289636357467, 0.4758125779702448, 0.431399393593233, 0.5653032341735655, 0.5172070068521097, 0.6133430758579169, 0.5028025818900566]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.035],0.01)

test_rmse_ave: 0.00904063613462884
[0.00901096281088072, 0.008700562033067225, 0.008974751757352439, 0.00955332661020307, 0.009691015613149412, 0.009329832348639322, 0.00850764185169428, 0.009372831123600349, 0.008034117194118081, 0.00923132000358353]


test_r2_ave: 0.519689485578201
[0.540550198289544, 0.5885768822944687, 0.5311976636746388, 0.46919415976264345, 0.4655354461060133, 0.43630995804768036, 0.5533941983113413, 0.5057271624488764, 0.6020351552682077, 0.5043740315785965]


## SVR

In [7]:
model=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)
svr_rmse_ave(skew_data,5,model)

train_rmse: 0.008794517024981087
test_rmse: 0.008955251354206354
test_r2: 0.5294414950675286


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 90, 
              max_depth = 3, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.5,
              reg_alpha =  0.05,
              reg_lambda = 0.1)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.009300400944874419
[0.009087498586840425, 0.008723248964934743, 0.00940858255489126, 0.01026415927878608, 0.009745186994810529, 0.00939392084996118, 0.008872040747426565, 0.01000530800665835, 0.008055483156951258, 0.00944858030748379]


test_r2_ave: 0.491103947356833
[0.5327122603709282, 0.5864284929307498, 0.48477933041493415, 0.38726426863048835, 0.45954358619009406, 0.4285391584859469, 0.514316870734511, 0.43676960229866235, 0.5999156421465206, 0.4807702613654944]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 6,
            min_samples_split = 4,
            max_leaf_nodes = 7,
            min_samples_leaf =9,
            random_state = 42) 
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.009366969856936084
[0.009272163264035575, 0.008726086437317532, 0.009301632127886646, 0.010225171544754072, 0.010111592235196634, 0.009248994583959785, 0.0092456694647097, 0.00987567232585224, 0.008028712088095762, 0.0096340044975529]


test_r2_ave: 0.48387783912984456
[0.5135280387408299, 0.5861593985622973, 0.49642611844168727, 0.39191030077859057, 0.41813877534716504, 0.4460357565614621, 0.47254830905518685, 0.4512702535938532, 0.6025704520655011, 0.4601909881518721]


In [9]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=5e-06)
elastic_mod=ElasticNet(alpha=[0.035], l1_ratio=0.01)
svr_mod=SVR(kernel='linear', C=1.3, gamma= 1e-07, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.08,
              booster = 'gbtree', 
              n_estimators = 90, 
              max_depth = 3, 
              min_child_weight = 4,
              seed = 42,
              gamma = 0,
              subsample = 0.8,
              colsample_bytree = 0.5,
              reg_alpha =  0.05,
              reg_lambda = 0.1)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 70,
            max_depth = 6,
            min_samples_split = 4,
            max_leaf_nodes = 7,
            min_samples_leaf =9,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])

# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.00901978215034669
[0.009013885756433455, 0.008452497533284824, 0.009009598989647842, 0.009632661044483397, 0.009614518084822986, 0.009268954140745387, 0.008597248943321777, 0.009447626519936948, 0.007870045073212614, 0.009290785417577678]


test_r2_ave: 0.5215264326986724
[0.5402520804095656, 0.6117028698933381, 0.5275500575840487, 0.460341529395505, 0.4739399007242032, 0.44364223990765694, 0.5439368659898869, 0.4978070707852895, 0.6181235967205524, 0.49796811557667753]


In [10]:

stack_mod = StackingRegressor(regressors=[svr_mod,elastic_mod,lr, lasso_mod], 
                           meta_regressor=vote_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.00897591643624758
[0.008978946352825135, 0.00841615395776593, 0.00889291242023024, 0.009616615741658164, 0.009574314627354554, 0.009371133471566118, 0.008449645345579345, 0.009346329765286288, 0.007882326722985894, 0.009230785957224132]


test_r2_ave: 0.5259671906976223
[0.543809300111116, 0.6150348476418495, 0.5397085494554916, 0.46213787037086385, 0.4783301813550056, 0.4313082472224993, 0.5594624581584207, 0.5085182899221117, 0.6169307873377881, 0.504431375401077]


In [11]:
ave(skew_data[title],y,0.5,stack_mod,0.3,vote_mod,0.2,svr_mod)

test_rmse_ave: 0.00896257904309267
[0.008997591657074581, 0.008337107865327235, 0.00894739984767813, 0.009533542349331405, 0.009530038405932847, 0.009297937827573686, 0.008517841994431943, 0.009370509426057204, 0.007828429430903278, 0.009265391626616385]


test_r2_ave: 0.5274042328140882
[0.5419127199401852, 0.6222322181003195, 0.5340508010183718, 0.47139040565863133, 0.4831439277547923, 0.44015737841121927, 0.5523226502012988, 0.5059719998710472, 0.622151531578081, 0.5007086956069362]


In [12]:
name='Input_A3_018'
test_pred=[]
seed=[1,6,8,2,0,5,9]
stack_w=0.5
vote_w=0.3
model_w=0.2
model=svr_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)


final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))
